In [1]:
import numpy as np
import ray
from tokenizers import Tokenizer
from functools import partial

In [2]:
dataset = ray.data.read_numpy('data/train', parallelism=1000)


In [3]:

class TokenToProbsProcessor:

    def __init__(self, rng, concentration: float, vocab_size: int):
        self.rng = rng
        self.concentration = concentration
        self.vocab_size = vocab_size

    def __call__(self, tokens):
        batch, seq_len, = tokens.shape

        def _tokens_to_probs(token_ids):
            x = self.rng.random((seq_len, self.vocab_size)) / self.vocab_size
            # At this point E(x.sum()) == 0.5 
            # What we want is for new_val / (x.sum() + new_val) ~ concentration
            # --> new_val == (concentration * x.sum())/(1 - concentration)
            # Then, in the normalized vector, the appropriate token will have ~ concentration weight,
            # and the others will have the rest
            x_sum = x.sum(axis=1)
            conc_val = np.mean((self.concentration * x_sum) / (1 - self.concentration))
            np.put_along_axis(x, token_ids[:, None], conc_val, axis=1)
            return x / x.sum(axis=1)[:, None]
            
        return np.apply_along_axis(_tokens_to_probs, axis=1, arr=tokens)
    
    
t2p = TokenToProbsProcessor(np.random.default_rng(332), 0.85, 8192)

        

In [6]:
pipe = dataset.window(blocks_per_window=1).map_batches(t2p, batch_size=16).iter_rows()

2022-06-18 23:41:33,998	INFO dataset.py:2969 -- Created DatasetPipeline with 1000 windows: 13.98MiB min, 194.57MiB max, 39.06MiB mean


In [7]:
next(pipe)

Stage 0:   0%|▏                                                                                                       | 2/1000 [01:20<11:07:42, 40.14s/it]


KeyboardInterrupt: 

2022-06-18 23:43:34,556	WARNING worker.py:1404 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ea9cf015c0bb94e2aa47ce19332c93abe827ed5601000000 Worker ID: 8e26079f9864f2d47452dee493e5f899af700938af39476eb9b61902 Node ID: ae89dc9bc9190f86b6e0c86077cd4599a988915babcdf12411e0ca7e Worker IP address: 127.0.0.1 Worker port: 55462 Worker PID: 10581
2022-06-18 23:44:33,541	WARNING worker.py:1404 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: b800d2eb7a21cc0f3e51352daeb55696aaf603e001000000 Worker ID: 865e75ecf6fca5814415ec3de30eeb585cd1ccd1fc74146f1bc6faf7 Node ID: ae89dc9bc9190f86b6e0c86077cd4599a988915babcdf12411e0ca7e Worker IP address: 127.0.0.1 Worker port: 55466 Worker PID: 10583
2022-06-18 23:45:30,568	WARNING worker.py:1404 -- A worker died or was killed while 